In [1]:
import torch
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import models

data_path = r"C:\Users\Arun\Downloads\images"

In [2]:
from torchvision import transforms

augmentation = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])


In [3]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader,random_split

train_data = ImageFolder(root=data_path, transform=augmentation)
training_len = int(len(train_data) * 0.8)
validation_len = len(train_data) - training_len
training_data,validation_data = random_split(train_data, [training_len, validation_len])
train_data_loader = DataLoader(train_data, batch_size=960, shuffle=True,pin_memory=True)
validation_data_loader = DataLoader(validation_data, batch_size=960, shuffle=True,pin_memory=True)

In [4]:
num_classes = len(train_data.classes)

In [9]:
from torchvision import models
from torch import nn

class cnn_model(nn.Module):
    def __init__(self, num_classes=101):
        super(cnn_model, self).__init__()
        self.base = models.mobilenet_v3_small(
            weights=models.MobileNet_V3_Small_Weights.IMAGENET1K_V1
        )

        for param in self.base.features.parameters():
            param.requires_grad = False

        infeatures = self.base.classifier[0].in_features
        self.base.classifier = nn.Sequential(
            nn.Linear(infeatures, 512),
            nn.Hardswish(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        return self.base(x)


In [10]:
model = cnn_model(num_classes=num_classes)
model_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer,"min",patience=2,factor=0.3)

In [11]:
from torch.amp import GradScaler,autocast

In [16]:
from tqdm import tqdm
device = ("cuda" if torch.cuda.is_available() else "cpu")
epochs = 4
scaler = GradScaler()
torch.backends.cudnn.benchmark =  True
model = model.to(device)
for epoch in range(epochs):
    model.train()
    running_loss, correct, total = 0, 0, 0

    loop = tqdm(train_data_loader, desc=f"Epoch [{epoch+1}/{epochs}]", leave=False)

    for image, label in loop:
        image, label = image.to(device), label.to(device)

        optimizer.zero_grad()

        with autocast(device_type="cuda",enabled=True,dtype= torch.float16):
            output = model(image)
            loss = model_loss(output, label)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()

        loop.set_postfix(loss=loss.item(), acc=f"{(correct/total)*100:.2f}%")

    train_loss = running_loss / len(train_data_loader)
    train_acc = (correct / total) * 100

    model.eval()
    val_loss, vcorrect, vtotal = 0, 0, 0
    with torch.no_grad():
        for image, label in validation_data_loader:
            image, label = image.to(device), label.to(device)
            with autocast(device_type="cuda"):
                output = model(image)
                loss = model_loss(output, label)

            val_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            vtotal += label.size(0)
            vcorrect += (predicted == label).sum().item()

    val_loss /= len(validation_data_loader)
    val_acc = (vcorrect / vtotal) * 100
    scheduler.step(val_loss)

    print(f"Epoch [{epoch+1}/{epochs}] | "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

print("Training complete.")

Epoch [1/4] | Train Loss: 1.5405 | Train Acc: 59.64% | Val Loss: 1.3866 | Val Acc: 64.22%


Epoch [2/4] | Train Loss: 1.5279 | Train Acc: 59.91% | Val Loss: 1.3540 | Val Acc: 64.80%


Epoch [3/4] | Train Loss: 1.5092 | Train Acc: 60.36% | Val Loss: 1.3421 | Val Acc: 65.50%


Epoch [4/4] | Train Loss: 1.4871 | Train Acc: 60.78% | Val Loss: 1.2901 | Val Acc: 65.93%
Training complete.


In [17]:
import pickle

pickle.dump(model,open("cnn_model","wb"))